You can download and run this notebook locally, or you can run it for free in a cloud environment using Colab or Sagemaker Studio Lab:

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kirbyju/TCIA_Notebooks/blob/main/TCIA_REST_API_Downloads.ipynb)

[![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_REST_API_Downloads.ipynb)

# Summary

Access to large, high quality data is essential for researchers to understand disease and precision medicine pathways, especially in cancer. However HIPAA constraints make sharing medical images outside an individual institution a complex process. [The Cancer Imaging Archive (TCIA)](https://www.cancerimagingarchive.net/) is a public service funded by the National Cancer Institute which addresses this challenge by providing hosting and de-identification services to take major burdens of data sharing off researchers.

**This notebook is focused on basic use cases for leveraging TCIA's REST APIs to query and download data.**  If you're interested in additional TCIA notebooks and coding examples check out https://github.com/kirbyju/TCIA_Notebooks.

# 1 Learn about available Collections on the TCIA website

[Browsing Collections](https://www.cancerimagingarchive.net/collections) and [Analysis Results](https://www.cancerimagingarchive.net/tcia-analysis-results/) datasets on TCIA are the easiest ways to become familiar with what is available.  These pages will help you quickly identify datasets of interest, find valuable supporting data that are not available via our APIs (e.g. clinical spreadsheets, non-DICOM segmentation data), and answer most common questions you might have about the datasets.  

# 2 REST API Overview
TCIA uses software called [NBIA](https://nbia.cancerimagingarchive.net/) to manage DICOM data.  You can find more information about its API on our Swagger page at https://cbiit.github.io/NBIA-TCIA/.  However, we will be leveraging the tcia_utils Python package in this notebook rather than accessing it directly.

# 3 Setup

The following cells install and import the **nbia** from [**tcia_utils**](https://pypi.org/project/tcia-utils/), which contains a variety of useful functions for accessing TCIA via Python.

Downloading data via **tcia_utils** is performed using the **downloadSeries()** function.  It works by specifying the DICOM Series Instance UIDs of the scans you'd like to download.  

By default, the function expects JSON data containing "SeriesInstanceUID" elements, which can be generated using any of the series-related queries such as **getSeries()** or **getCart()**.  However, if you have a series UID list from some other source, you can set **input_type = "list"** to pass a python list of one or more series UIDs instead of JSON. You can also set **input_type = "manifest"** to provide a path to a **.TCIA** manifest file.

Data are saved to a **tciaDownload** folder in your current working directory by default, but you can use the **path** parameter to change this to a different directory.  Data are unzipped into a folder named after the Series Instance UID, unless `as_zip` is set to True.  In this case, it skips the unzipping steps and names the zip file after the Series Instance UID.  

There is an optional **format** parameter that can be used to return metadata about what was downloaded.  It can be set to **df** to return dataframe or **csv** to save a spreadsheet. There's also a **csv_filename** parameter if you want to set a specific file name.

You can specify **number = n** to tell the function to only download the first **n** scans of your seriesUids.  Remove this parameter in the examples below to download the full dataset.

The **api_url** parameter can be omitted in most cases.  However, it must be set to **api_url = "nlst"** to access the [National Lung Screening Trial (NLST)](https://doi.org/10.7937/TCIA.HMQ8-J677) collection.  

Last but not least, there is some logic built in to detect whether you've already downloaded a series.  If a directory named after the seriesUid already exists the function will assume it's already been downloaded and skip that series.

In [ ]:
import sys

# install tcia utils
!{sys.executable} -m pip install --upgrade -q tcia_utils

In [ ]:
import requests
import pandas as pd
from tcia_utils import nbia_v4 as nbia

# set logging level to INFO in Google Colab (not necessary in Jupyter)
if 'google.colab' in sys.modules:
  import logging

  for handler in logging.root.handlers[:]:
      logging.root.removeHandler(handler)

  # Set handler with level = info
  logging.basicConfig(format='%(asctime)s:%(levelname)s:%(message)s',
                      level=logging.INFO)

  print("Google Colab Logging set to INFO")

# 4 Download Examples

In this section we'll cover downloading data via the REST API for the following use cases:

1.   Download a full TCIA collection
2.   Download custom results of an API query
3.   Download a "[Shared Cart](https://wiki.cancerimagingarchive.net/display/NBIA/TCIA+Radiology+Portal+User+Guide#TCIARadiologyPortalUserGuide-SharingDatainYourCart)" that was created via https://nbia.cancerimagingarchive.net/
4.   Download data from a TCIA manifest file
5.   Download data from a **restricted** collection that requires logging in with your account
6.   Download individual images from a series

## 4.1 Download a full collection

You can [Browse Collections](https://www.cancerimagingarchive.net/collections) on our website to figure out what you might want to download, but you can also get a list of available collections via the API as shown below.

In [ ]:
# get list of available collections as JSON
nbia.getCollections()

Let's say that we're interested in downloading the entire **Soft-tissue-Sarcoma** collection.  First we need to get a list of all Series Instance UIDs in that collection.  We can use **nbia.getSeries()** to return JSON metadata about all series (scans) in this collection.

In [ ]:
data = nbia.getSeries(collection = "Soft-tissue-Sarcoma")
print(data)

We can then pass that to the our download functions and view/save the metadata for what was downloaded.  We'll leverage the **number** parameter here to just grab the first 2 scans as a test.  You can remove this parameter if you want to download the full collection.

In [ ]:
nbia.downloadSeries(data, number = 2)

Take a second to go look at your **tciaDownload** folder to view the data.  Note that each series is saved in a directory named by its Series Instance UID.

## Optional -- View an example series in the notebook
You can learn more about various ways to visualize and analyze your data in the other notebooks at https://github.com/kirbyju/TCIA_Notebooks. If you don't have a DICOM viewer handy, you can use **dicomViewer.viewSeries()** from the [simpleDicomViewer](https://pypi.org/project/simpleDicomViewer/) package to get a quick look at one of the series we've downloaded. You can change the Series UID in the code below to view other scans you've downloaded.

**Note:** Installation may take a couple of minutes.


In [ ]:
# OPTIONAL -- if you want to look at the images and don't have another viewer installed
!{sys.executable} -m pip install --upgrade -q simpleDicomViewer

from simpleDicomViewer import dicomViewer

In [ ]:
seriesUid = "1.3.6.1.4.1.14519.5.2.1.5168.1900.104193299251798317056218297018"
dicomViewer.viewDicom(f"tciaDownload/{seriesUid}")

## 4.2 Download custom API query
The REST API allows for a variety of different query options.  Additional examples are demonstrated in [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_REST_API_Queries.ipynb).  

### 4.2.1 Custom downloads with getSeries()

For this use case, let's assume that you are only interested in the MR scans from the [TCGA-BRCA](https://doi.org/10.7937/K9/TCIA.2016.AB2NAZRP) collection that were acquired on Siemens scanners.

In [ ]:
# getSeries with query parameters
data = nbia.getSeries(collection = "TCGA-BRCA",
                      modality = "MR",
                      manufacturer = "SIEMENS")

print(len(data), 'Series returned')

Once again, let's pass those Series Instance UIDs to our download function.  This time we'll also set **format = "df"** to return a dataframe about what we downloaded.

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, number = 2, format = "df")
display(df)

Another common use case may be that you'd like to review the results from **getSeries()** and do some further filtering.  This time we'll add **format = "df"** to save the output to a dataframe.  After removing unwanted scans from the dataframe, we'll pass the remaining Series Instance UIDs to **downloadSeries()**.  For example, let's say that you only wanted to download **T2 MR** series instead of any MR.

In [ ]:
# getSeries with query parameters
df = nbia.getSeries(collection = "TCGA-BRCA",
               modality = "MR",
               manufacturer = "SIEMENS", format = "df")

Let's drop scans that don't contain **t2** in either the **ProtocolName** or **SeriesDescription** fields.  Note that we're down to 21 scans now instead of 353.

In [ ]:
# convert the columns to lowercase to catch both upper/lower case with filter for 't2'
filtered_df = df[(df['ProtocolName'].str.lower().str.contains('t2')) |
                 (df['SeriesDescription'].str.lower().str.contains('t2'))]

display(filtered_df)

Now we can feed the Series Instance UIDs from this dataframe back to **downloadSeries()**.  Note that since we're not working with the API's default JSON output anymore we need to specify **input_type = "df"** when we call **downloadSeries()**.

Let's also try out the `as_zip` parameter this time just for fun.

In [ ]:
# download the selected series_uids
nbia.downloadSeries(filtered_df, input_type = "df", number = 2, as_zip=True)

### 4.2.2 Custom downloads with  getSimpleSearchWithModalityAndBodyPartPaged()
This function lets you do pretty much everything you can do in the **Simple Search** screen at https://nbia.cancerimagingarchive.net. As as a result, there are a lot of parameters.

```
collections: list[str]   -- The DICOM collections of interest to you
    species: list[str]       -- Filter collections by species. Possible values are 'human', 'mouse', and 'dog'
    modalities: list[str]    -- Filter collections by modality
    modalityAnded: bool      -- If true, only return subjects with all requested modalities, as opposed to any
    minStudies: int          -- The minimum number of studies a collection must have to be included in the results
    manufacturers: list[str] -- Imaging device manufacturers, e.g. SIEMENS
    bodyParts: list[str]     -- Body parts of interest, e.g. CHEST, ABDOMEN
    fromDate: str            -- First cutoff date, in YYYY/MM/DD format. Defaults to 1900/01/01
    toDate: str              -- Second cutoff date, in YYYY/MM/DD format. Defaults to today's date
    patients: list[str]      -- Patients to include in the output
    start: int               -- Starting point of returned subject results. Defaults to 0.
    size: int                -- Number of returned subjects per page. Defaults to 10.
    sortDirection            -- 'ascending' or 'descending'. Defaults to 'ascending'.
    sortField                -- 'subject', 'studies', 'series', or 'collection'. Defaults to 'subject'.
    format: str              -- Defaults to JSON. Can be set to "uids" to return a python list of
                                Series Instance UIDs or "manifest" to save a TCIA manifest file (up to 1,000,000 series).
```

By default it returns paged results in JSON format, but you can also set it to **manifest** or **uids** to return the results in a way that makes it easy to download them with `nbia.downloadSeries()`.  You just need to set the `input_type` parameter of `downloadSeries()` and the `format` parameter of `getSimpleSearchWithModalityAndBodyPartPaged()` accordingly.  For example:

In [ ]:
uids = nbia.getSimpleSearchWithModalityAndBodyPartPaged(bodyParts=["CHEST"], modalities=["CT"], format = "uids")

In [ ]:
# just grab the first series using number = 1 for demonstration purposes
nbia.downloadSeries(uids, input_type = "list", number = 1)

Another common use case might be that you want to download a specific set of Patient IDs.  You can combine this with other query parameters as shown below.  Let's say you only want chest xrays for these two LIDC subjects:

In [ ]:
# set patient ID list
ids = ['LIDC-IDRI-0001', 'LIDC-IDRI-0002']

# submit query
uids = nbia.getSimpleSearchWithModalityAndBodyPartPaged(patients=ids, modalities = ["CR", "DX"], format = "uids")

# download images
nbia.downloadSeries(uids, input_type = "list", format = "df")

### 4.2.3 Download custom NLST API query
Let's show a similar example where we look for a specific modality and manufacturer within the [National Lung Screening Trial (NLST) Collection](https://doi.org/10.7937/TCIA.HMQ8-J677).  We have to set **api_url = "nlst"** in our functions for this to work, but otherwise the steps are the same.

In [ ]:
# getSeries with query parameters
data = nbia.getSeries(modality = "CT",
                      manufacturer = "Philips",
                      api_url = "nlst")

print(len(data), 'Series returned')

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, number = 2, api_url = "nlst", format = "df")
display(df)

## 4.3 Download a "shared cart"
It's possible to use https://nbia.cancerimagingarchive.net to create a "[Shared Cart](https://wiki.cancerimagingarchive.net/display/NBIA/TCIA+Radiology+Portal+User+Guide#TCIARadiologyPortalUserGuide-SharingDatainYourCart)" which includes a specific set of scans you'd like to share with others.

After creating a Shared Cart you will receive a URL that looks like https://nbia.cancerimagingarchive.net/nbia-search/?saved-cart=nbia-49121659384603347 which can be shared with others.  Try clicking the link to see what this looks like on the TCIA website.  Then use the code below to see how you can use the cart name at the end of the URL to download the related scans via the API.

In [ ]:
# getSharedCart metadata
data = nbia.getSharedCart(name = "nbia-49121659384603347")
print(len(data), 'Series returned')

We'll skip the use of the **number** parameter this time since the full cart is only 4 series.  Let's also try **format = "csv"** to save a spreadsheet of the metadata in addition to returning a dataframe.

In [ ]:
# feed series_data to our downloadSampleSeries function
df = nbia.downloadSeries(data, format = "csv")
display(df)

## 4.4 Download data from a TCIA manifest file

When working with manifest files you can install the NBIA Data Retriever to open the manifest and download the data as shown in [this notebook](https://github.com/kirbyju/TCIA_Notebooks/blob/main/TCIA_Linux_Data_Retriever_App.ipynb).  However, there may be cases where you don't have administrative rights to install software or prefer using the REST API to download a manifest.  

In order to demonstrate this use case, let's assume that after you [Browse Collections](https://www.cancerimagingarchive.net/collections) you are interested in the [RIDER Breast MRI](https://doi.org/10.7937/K9/TCIA.2015.H1SXNUXL) collection.  We can find the URL of the manifest to download the full collection by looking at the blue "Download" button on that page.  Then we can download the manifest with the following commands.  

In [ ]:
# download manifest file from RIDER Breast MRI page
manifest = requests.get("https://www.cancerimagingarchive.net/wp-content/uploads/doiJNLP-Fo0H1NtD.tcia")
with open('RIDER_Breast_MRI.tcia', 'wb') as f:
    f.write(manifest.content)

If you open this manifest file in a text editor you'll notice that it contains several lines of download parameters that precede a list of Series Instance UIDs to download.  If we set **input_type = "manifest"** we can provide the path/filename to **downloadSeries()** and it will extract the UIDs from the file and download them.

In [ ]:
df = nbia.downloadSeries("RIDER_Breast_MRI.tcia", input_type = "manifest", number = 2, format = "df")
display(df)

## 4.6 Download individual images from a series

It is also possible to download specific images if you don't want the entire series. Let's look at an example using the [**CMB-GEC**](https://doi.org/10.7937/E7KH-R486) collection.  First we'll get a list of the metadata about the series.

In [ ]:
nbia.getSeries(collection = "CMB-GEC")

Once we identify a series of interest we can pass its Series Instance UID to **nbia.getSopInstanceUids()** to obtain a list of SOP Instance UIDs for the individual images that are part of this series.

In [ ]:
seriesUID = "1.3.6.1.4.1.14519.5.2.1.1600.1204.919741553251398079475267746505"

nbia.getSopInstanceUids(seriesUid)

After we have both a series UID and SOP Instance UID, we can call the **nbia.downloadImage()** function to download a specific image from the series.

In [ ]:
sopUID = "1.3.6.1.4.1.14519.5.2.1.1600.1204.211684247543622814130853101548"

nbia.downloadImage(seriesUID, sopUID)

# Acknowledgements
TCIA is funded by the [Cancer Imaging Program (CIP)](https://imaging.cancer.gov/), a part of the United States [National Cancer Institute (NCI)](https://www.cancer.gov/).  It is managed by the [Frederick National Laboratory for Cancer Research (FNLCR)](https://frederick.cancer.gov/) and hosted by the [University of Arkansas for Medical Sciences (UAMS)](https://www.uams.edu/)

This notebook was created by [Justin Kirby](https://www.linkedin.com/in/justinkirby82/). If you leverage this notebook or any TCIA datasets in your work, please be sure to comply with the [TCIA Data Usage Policy](https://wiki.cancerimagingarchive.net/x/c4hF). In particular, make sure to cite the DOI(s) for the specific TCIA datasets you used in addition to the following paper!

## TCIA Citation

Clark, K., Vendt, B., Smith, K., Freymann, J., Kirby, J., Koppel, P., Moore, S., Phillips, S., Maffitt, D., Pringle, M., Tarbox, L., & Prior, F. (2013). The Cancer Imaging Archive (TCIA): Maintaining and Operating a Public Information Repository. Journal of Digital Imaging, 26(6), 1045–1057. https://doi.org/10.1007/s10278-013-9622-7